In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rioxarray
import xarray
from rioxarray.merge import merge_arrays
import os

In [2]:
bucket_name = 'opera-pst-rs-pop1'
local_path = "C:/Users/Matthew Bonnema/Documents/dswx_data/wtr/"
local_dest = "C:/Users/Matthew Bonnema/Documents/dswx_data/mosaic/"

In [3]:
with open('dswx_manifest.txt') as f:
    lines = f.readlines()

print(lines[0])

s3://opera-pst-rs-pop1/products/OPERA_L3_DSWx_HLS_T01GDM_20220101T214749Z_20221125T202742Z_S2B_30_v0.0/



In [4]:
cont = 'af' # af,as,au,na,sa,eu
lvl = '03'

In [5]:
basin_path  = f'./shapefiles/basins/hybas_{cont}/hybas_{cont}_lev{lvl}_v1c.shp'
basin_gdf = gpd.read_file(basin_path)
basin_gdf.head()

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,geometry
0,1030000010,0,1030000010,1030000010,0.0,0.0,236343.2,236343.2,111,0,1,0,1,"MULTIPOLYGON (((39.76528 15.42917, 39.75492 15..."
1,1030003990,0,1030003990,1030003990,0.0,0.0,519027.5,519027.5,112,0,1,0,2,"MULTIPOLYGON (((40.81528 14.75417, 40.79561 14..."
2,1030008100,0,1030008100,1030008100,0.0,0.0,797881.4,797881.4,114,0,0,1,3,"POLYGON ((36.74167 4.06667, 36.74131 4.06909, ..."
3,1030008110,0,1030008110,1030008110,0.0,0.0,1040194.7,1040194.7,117,0,1,0,4,"MULTIPOLYGON (((40.84444 -2.43333, 40.83833 -2..."
4,1030011530,0,1030011530,1030011530,0.0,0.0,4421.4,4421.4,121,0,1,0,5,"POLYGON ((36.42083 -18.56667, 36.42083 -18.570..."


In [6]:
mrg_gdf = gpd.read_file('./shapefiles/world_mgrs/mgrs_region.shp') 
mrg_gdf.head()

,GRID1MIL,GRID100K,LONGITUDE,LATITUDE,geometry
0,33L,SC,12.102867,-15.678288,"POLYGON ((12.00000 -16.00000, 12.00000 -15.909..."
1,33L,TC,12.666855,-15.679626,"POLYGON ((13.13098 -16.00000, 13.12500 -16.000..."
2,33L,UC,13.599601,-15.684492,"POLYGON ((14.06538 -16.00000, 14.06250 -16.000..."
3,33L,VC,14.532691,-15.687415,"POLYGON ((15.00000 -16.00000, 14.90625 -16.000..."
4,33L,WC,15.467309,-15.687415,"POLYGON ((15.93462 -16.00000, 15.84375 -16.000..."


In [7]:
def make_mosaic(basinID):
    try:
        mosaics_list = os.listdir(local_dest)

        if f'{cont}_basin{basinID}_WTR.tif' in mosaics_list:
            print(f'mosaic alreaddy exists for basin {basinID}')
            return
        target_basin =  basin_gdf[basin_gdf.PFAF_ID==basinID]
        grid_codes = gpd.sjoin(mrg_gdf,target_basin)
        
        tile_list = []
        for row in grid_codes.iterrows():
            tile_list.append(row[1].iloc[0]+row[1].iloc[1])
        
        keys = []
        for line in lines:
            prefix = line.split(bucket_name+'/')[-1][:-2]
            filename = prefix.split('/')[-1]+'_B01_WTR.tiff'
            tile = filename.split('_')[4][1:]
            if tile in tile_list:
                keys.append(filename)
        
        if len(keys) == 0:
            print(f'no prodducts found for basin: {basinID}')
        elif f'{cont}_basin{basinID}_WTR.tif' in mosaics_list:
            print(f'mosaic alreaddy exists for basin {basinID}')
        else:
            ds_to_mosaic = []
            i = 0
            for key in keys:
                file_path = f'{local_path}{key}'
                xds = rioxarray.open_rasterio(file_path,cache=False)
                reproj = xds.rio.reproject("EPSG:4326")
                ds_to_mosaic.append(reproj)
                xds.close()
                reproj.close()
                del xds
                del  reproj

            merged_raster =  merge_arrays(dataarrays = ds_to_mosaic, crs="EPSG:4326", nodata = 255)
            merged_raster.rio.to_raster(f'{local_dest}{cont}_basin{basinID}_WTR.tif', tiled=True, windowed=True)
            merged_raster.close()
            del merged_raster
            print(f'completed mosaic for basin {basinID}')
    except Exception as e:
        print(f'failed to make mosaic for basin {basinID}, error: {e}')

In [8]:
for row in basin_gdf.iterrows():
    ID = row[1]['PFAF_ID']
    make_mosaic(ID)

mosaic alreaddy exists for basin 111
mosaic alreaddy exists for basin 112
mosaic alreaddy exists for basin 114
mosaic alreaddy exists for basin 117
mosaic alreaddy exists for basin 121
mosaic alreaddy exists for basin 122
mosaic alreaddy exists for basin 123
mosaic alreaddy exists for basin 124
mosaic alreaddy exists for basin 125
mosaic alreaddy exists for basin 126
mosaic alreaddy exists for basin 127
mosaic alreaddy exists for basin 128
mosaic alreaddy exists for basin 131
mosaic alreaddy exists for basin 132
mosaic alreaddy exists for basin 133
mosaic alreaddy exists for basin 141
mosaic alreaddy exists for basin 142
mosaic alreaddy exists for basin 143
mosaic alreaddy exists for basin 144
completed mosaic for basin 145
completed mosaic for basin 146
completed mosaic for basin 147


KeyboardInterrupt: 